In [223]:
import pandas as pd

def read_csv_column(csv_path, column_name):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    return df[column_name].tolist()

In [224]:
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

contents = read_csv_column('./trump/trump_10.csv', '微博正文')

documents = []

for content in contents:
    content = content.split()
    new_content = []
    for i in content:
        if len(i) != 1:
            new_content.append(i)
    documents.append(new_content)

print(documents[:2])
logging.info ("Done reading data file")

2020-12-23 16:07:26,169 : INFO : Done reading data file


[['占豪', '观点', '同比', '下降', '314', '今年', '美帝', '经济', '特朗普', '辩论', '经济', '增长', '增长', '几十', '百分点', '有时候', '危机', '追赶', '时间', '20', '经济体', '基本', '中国', '增长', '别国', '暴跌', '314', '美国', '季度', '实际', 'GDP', '跌幅'], ['美国', '美国', '中国', '卡脖子', '火力', '扶持', '稀土', '工业', '美国', '总统', '特朗普', '9月', '30日', '中国', '威胁', '借口', '签署', '行政', '命令', '命令', '内阁', '部长', '调查', '美国', '中国', '稀土', '依赖', '特朗普', '旨在', '结束', '中国', '稀土', '行业', '主导', '地位', '最新', '动作']]


In [225]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
# model.train(documents,total_examples=len(documents),epochs=10)

2020-12-23 16:07:26,175 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-12-23 16:07:26,176 : INFO : collecting all words and their counts
2020-12-23 16:07:26,177 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-12-23 16:07:26,214 : INFO : PROGRESS: at sentence #10000, processed 192148 words, keeping 12214 word types
2020-12-23 16:07:26,263 : INFO : PROGRESS: at sentence #20000, processed 451916 words, keeping 19690 word types
2020-12-23 16:07:26,305 : INFO : PROGRESS: at sentence #30000, processed 688209 words, keeping 24436 word types
2020-12-23 16:07:26,363 : INFO : PROGRESS: at sentence #40000, processed 1026468 words, keeping 32412 word types
2020-12-23 16:07:26,437 : INFO : PROGRESS: at sentence #50000, processed 1387794 words, keeping 38883 word types
2020-12-23 16:07:26,448 : INFO : collected 39918 word types from a corpus of 1445428 raw words and 51814 sentences
2020-12-23 16:07:26,448 : INFO : Loading a fre

In [226]:
from sklearn.cluster import KMeans

#获取model里面的说有关键词
vocab = model.wv.vocab
print(type(vocab))

<class 'dict'>


In [227]:
keys=model.wv.vocab.keys()

#获取词对于的词向量
word_vector=[]
for key in keys:
    word_vector.append(model[key])

<ipython-input-227-e64ffd61c5c2>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word_vector.append(model[key])


In [228]:
print(len(word_vector[0]))

150


In [229]:
#分类
kMeans_clustering = KMeans(n_clusters=10, init='k-means++')
clusters = kMeans_clustering.fit_predict(word_vector)
centers = kMeans_clustering.cluster_centers_
centroid_map = dict(zip(model.wv.index2word, clusters))
print(centroid_map)
print(len(centroid_map))
# s = clf.fit(word_vector)

{'特朗普': 0, '美国': 3, '新冠': 4, '总统': 8, '确诊': 0, '视频': 5, '哈哈哈': 3, '病毒': 4, '夫妇': 7, '检测': 5, '白宫': 4, '朗普': 2, '中国': 4, '感染': 3, '接受': 9, '拜登': 0, '阳性': 7, '大选': 4, '竞选': 2, '时间': 3, '疫情': 9, '症状': 2, '表示': 4, '治疗': 5, '出现': 4, '辩论': 2, '没有': 4, '10月': 4, '报道': 2, '可能': 2, '政府': 2, '当地': 6, '医生': 9, '消息': 7, '肺炎': 7, '发烧': 1, '助理': 9, '疗法': 9, '抗体': 9, '国家': 9, '口罩': 9, '问题': 9, '希望': 9, '媒体': 9, '2日': 9, '良好': 9, '深圳': 5, '现在': 4, '已经': 9, '医院': 9, '鸡尾酒': 7, '隔离': 2, '目前': 9, '第一夫人': 4, '集会': 3, '进行': 5, '围观': 0, '美元': 2, '中心': 3, '活动': 4, '世界': 5, '梅拉尼娅': 5, '前往': 3, '除恶': 2, '发布': 2, '状况': 9, '康复': 7, '民主党': 4, '军事': 0, '投票': 4, '市府': 9, '候选人': 2, '公司': 2, '记者': 4, '回应': 3, '身体': 3, '宣布': 4, '经济': 2, '夫人': 1, '知道': 1, '参加': 4, '全球': 1, '医疗': 2, '举行': 1, '计划': 2, '认为': 2, '市场': 9, '大法官': 4, '选举': 5, '连任': 9, '工作': 9, '情况': 7, '阴性': 2, '川普': 9, '第一': 9, '继续': 9, '发生': 9, '非常': 9, '刚刚': 2, '经理': 9, '证实': 2, '今天': 9, '势力': 2, '巴雷特': 8, 'CNN': 2, '早日': 2, '影响': 3, '社交': 0, '举报': 3, '团队':

In [230]:
from sklearn.neighbors import KDTree
import numpy as np

def get_top_words(index2word, k, word_centers, word_vectors):
    tree = KDTree(word_vectors)
# Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k = k) for x in word_centers]
    closest_words_idxes = [x[1] for x in closest_points]
# Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {}
    for index in range(0, len(closest_words_idxes)):
        closest_words['Cluster #' + str(index)] = [index2word[j] for j in closest_words_idxes[index][0]]
# A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words)
    df.index = df.index + 1
    return df

In [231]:
import openpyxl
Z = model.wv.syn0
top_words = get_top_words(model.wv.index2word, 20, centers, Z)

<ipython-input-231-f78fd8717bd7>:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  Z = model.wv.syn0


In [232]:
top_words.to_excel("./trump/trump10.xlsx")
#获取到所有词向量所属类别
# labels=clf.labels_
#
# #把是一类的放入到一个集合
# #把是一类的放入到一个字典里
# classCollects={}
# for i in range(len(keys)):
#
#     if labels[i] in list(classCollects.keys()):
#         classCollects[labels[i]].append(list(keys)[i])
#     else:
#         classCollects={0:[],1:[],2:[],3:[],4:[],5:[],6:[],7:[],8:[],9:[]}
#
# print('0类：',classCollects[0])
# print('1类：',classCollects[1])
# print('2类：',classCollects[2])
# print('3类：',classCollects[3])
# print('4类：',classCollects[4])
# print('5类：',classCollects[5])
# print('6类：',classCollects[6])
# print('7类：',classCollects[7])
# print('8类：',classCollects[8])
# print('9类：',classCollects[9])